<a href="https://colab.research.google.com/github/dani-lbnl/mudit/blob/main/Topic_Modeling_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install packages

In [1]:
!pip install bertopic

     |████████████████████████████████| 57 kB 1.8 MB/s 
     |████████████████████████████████| 79 kB 5.8 MB/s 
     |████████████████████████████████| 5.2 MB 12.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 86 kB 6.1 MB/s 
     |████████████████████████████████| 4.0 MB 41.0 MB/s 
     |████████████████████████████████| 1.2 MB 47.7 MB/s 
     |████████████████████████████████| 77 kB 7.5 MB/s 
     |████████████████████████████████| 6.5 MB 45.9 MB/s 
     |████████████████████████████████| 636 kB 47.2 MB/s 
     |████████████████████████████████| 895 kB 42.1 MB/s 
     |████████████████████████████████| 1.1 MB 42.4 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2330770 sha256=8c3baaeb0f3aebe8ddcb626e44d696ccc26973738037d70d68823d814b27bdef
  Stored in directory: /root/.cache/pip/wheels/6e/7a/5e/259ccc841c085fc41b99ef

In [17]:
# from top2vec import Top2Vec
import pandas as pd
import numpy as np
import multiprocessing
import time
from google.colab import drive
import os
from bertopic import BERTopic
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')

multiprocessing.cpu_count()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


2

## Import data from Google Drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
# read all files
path = '/content/drive/MyDrive/NLP/ALS Spreadsheets/'
files = os.listdir(path)
files.sort()
files
# beamline 2.4 bad ..removed; also one more check which one
# 3.2.1 bad

['Beamline_1.4.xls',
 'Beamline_10.0.1.xls',
 'Beamline_10.3.1.xls',
 'Beamline_10.3.2.xls',
 'Beamline_11.0.1.xls',
 'Beamline_11.0.2.xls',
 'Beamline_11.3.1.xls',
 'Beamline_11.3.2.xls',
 'Beamline_12.0.1.1.xls',
 'Beamline_12.0.1.2.xls',
 'Beamline_12.0.1.4.xls',
 'Beamline_12.0.2.xls',
 'Beamline_12.2.1.xls',
 'Beamline_12.2.2.xls',
 'Beamline_12.3.1.xls',
 'Beamline_12.3.2.xls',
 'Beamline_2.1.xls',
 'Beamline_3.1.1.xls',
 'Beamline_3.2.1.xls',
 'Beamline_3.3.2.xls',
 'Beamline_4.0.2.xls',
 'Beamline_4.0.3.1.xls',
 'Beamline_4.0.3.2.xls',
 'Beamline_4.2.2.xls',
 'Beamline_5.0.1.xls',
 'Beamline_5.0.2.xls',
 'Beamline_5.0.3.xls',
 'Beamline_5.3.1.xls',
 'Beamline_5.3.2.1.xls',
 'Beamline_5.3.2.2.xls',
 'Beamline_5.4.xls',
 'Beamline_6.0.1.xls',
 'Beamline_6.0.2.xls',
 'Beamline_6.1.2.xls',
 'Beamline_6.3.1.xls',
 'Beamline_6.3.2.xls',
 'Beamline_7.0.1.xls',
 'Beamline_7.0.2.xls',
 'Beamline_7.3.1.xls',
 'Beamline_7.3.3.xls',
 'Beamline_8.0.1.xls',
 'Beamline_8.2.1.xls',
 'Beamline_

In [19]:
def create_input(df):

  # for title
  indices = df['Title'].isna()
  df.loc[indices,'Title'] = ""

  # for abstract
  indices = df['Abstract'].isna()
  df.loc[indices,'Abstract'] = ""

  # combined - title + abstract
  df['Combined'] = df['Title'] + " " + df['Abstract']
  combined = list(df['Combined'])

  # remove elements with blank
  while " " in combined:
    combined.remove(" ")

  # remove patterns
  pattern = r'<inf>|</inf>|<sup>|</sup>'
  comb_clean = []
  for l in combined:
    mod_string = re.sub(pattern, '', l )
    comb_clean.append(mod_string)


  return comb_clean



In [6]:
def remove_stop(df_combined):

  # remove stopwords

  new_df= []
  for doc in df_combined:
    filt_combined = []
    for word in word_tokenize(doc):
      # print(word)
      if word not in stopwords.words('english'):
        # print(word)
        filt_combined.append(word)
    new_df.append(" ".join(filt_combined))

  return new_df

In [7]:
def berttopic_basic(df):

  topic_model = BERTopic()
  topics, probs = topic_model.fit_transform(df)

  # topic_info_df = pd.DataFrame(topic_model.get_topic_info())

  return topic_model

In [8]:
import joblib
import os 

def create_path_if_not_exists(datapath):
    '''Create the new file if not exists and save the data'''

    if not os.path.exists(datapath):
        os.makedirs(datapath) 

In [16]:
for i in range(len(files)):
  print("File:",str(i+1) + ' ' + files[i])
  beam_name = files[i].split('xls')[0][:-1]

  df = pd.read_table(path + files[i], on_bad_lines='skip')


  # combine abstract and title
  input_data = create_input(df)

  # remove stopwords
  filtered_input = remove_stop(input_data)

  # run topic modeling algorithm
  model = berttopic_basic(filtered_input)

  base_path = '/content/drive/MyDrive/NLP/'
  model_path = base_path + "BERTopic_v1/" + beam_name + "/"
  create_path_if_not_exists(model_path)

  # save model as pickle file
  file_path = model_path + "model" + beam_name + ".pkl"
  joblib.dump(model, file_path) 

  # visualize as html
  fig = model.visualize_barchart(top_n_topics = len(model.topics))
  fig_name = model_path + "bar_chart.html"
  fig.write_html(fig_name)

  # save as excel
  df_info = pd.DataFrame(model.get_topic_info())
  excel_name = model_path + "Topic_Results.xlsx"
  df_info.to_excel(excel_name,sheet_name="topic_info")

  df_topics = pd.DataFrame(model.topics)
  df_topics.to_excel(excel_name,sheet_name="topic_words")

  print("File: {} done ".format(i+1) )


File: 1 Beamline_1.4.xls
Length (before removing blanks): 369
Length (after removing blanks): 368
File: 1 done 
File: 2 Beamline_10.0.1.xls
Length (before removing blanks): 1059
Length (after removing blanks): 1059
File: 2 done 
File: 3 Beamline_10.3.1.xls
Length (before removing blanks): 117
Length (after removing blanks): 117
File: 3 done 
File: 4 Beamline_10.3.2.xls
Length (before removing blanks): 552
Length (after removing blanks): 549
File: 4 done 
File: 5 Beamline_11.0.1.xls
Length (before removing blanks): 1105
Length (after removing blanks): 1102
File: 5 done 
File: 6 Beamline_11.0.2.xls
Length (before removing blanks): 794
Length (after removing blanks): 792
File: 6 done 
File: 7 Beamline_11.3.1.xls
Length (before removing blanks): 1077
Length (after removing blanks): 1076
File: 7 done 
File: 8 Beamline_11.3.2.xls
Length (before removing blanks): 193
Length (after removing blanks): 193
File: 8 done 
File: 9 Beamline_12.0.1.1.xls
Length (before removing blanks): 290
Length (af

KeyboardInterrupt: ignored